# Configuration

## Modules

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from sklearn.pipeline import Pipeline
import pandas as pd
import re

## Constants

In [ ]:
MAIN_URL = 'https://www.dailymail.co.uk/news/breaking_news/index.html'
DOMAIN = 'https://www.dailymail.co.uk'
SCRAPE_OUTPUT_FILE = '/content/drive/MyDrive/MA5851_A3/scrape_results.csv'
ARTICLE_TAG = 'a'
URL_HTML_TAG = 'href'
URLS_START_WITH = ['https://www.dailymail.co.uk']
URLS_NOT_END_WITH = ['#video']
TEXT_TAG = 'p'
REMOVE_SUBSTRINGS = ["]",'"',"'",".",",","[","/",">",]
SEED = 42

# Functions

## Utilities

In [ ]:
def Set_All_Seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    np.random.RandomState(seed)

## HTML Element Selection

In [ ]:
def Get_Soup(url: str):
  data = requests.get(url)
  html = BeautifulSoup(data.text, 'html.parser')
  return html

In [ ]:
def Get_Links(soup: BeautifulSoup, find_tag: str, get_tag: str):
  results = []
  for link in soup.find_all(find_tag):
    results.append(link.get(get_tag))
  return results

In [ ]:
def Get_Content(soup: BeautifulSoup, tag: str):
  results = []
  for p in soup.find_all(tag):
    results.append(p.contents)
  return results

## Link Selection

In [ ]:
def Select_Links_Starts_With(links: list, stem: str):
  results = []
  for link in links:
    if not isinstance(link, str):
      continue
    if link.startswith(stem):
      results.append(link)
  return results

def Select_Links_Ends_With(links: list, stem: str):
  results = []
  for link in links:
    if not isinstance(link, str):
      continue
    if link.endswith(stem):
      results.append(link)
  return results

def Remove_Links(func, links: list, stems: list):
  for s in stems:
    delta = func(links = links, stem=s)
    links = list(set(links) - set(delta))
  return links

def Append_Links(func, links: list, stems: list):
  results = []
  for s in stems:
    delta = func(links = links, stem=s)
    results.append(delta)
  return results[0]

## Text Processing

In [ ]:
def Remove_Sub_Strings(string: str, remove: list):
  for r in remove:
    assert isinstance(string, str)
    string = string.replace(r,"")
  return string

def Clean_String(s: str):
  return re.sub('(^|\s+)FIRST($|\s+)', '', s)

def Get_Text_From_Page(url: str):
  web_text = Get_Content(soup = Get_Soup(url), tag=TEXT_TAG)
  return Remove_Sub_Strings(str(web_text), remove = REMOVE_SUBSTRINGS)

# Execute Pipeline

In [ ]:
Set_All_Seeds(SEED)

In [ ]:
URLs = Get_Soup(MAIN_URL)
URLs = Get_Links(soup = URLs, find_tag = ARTICLE_TAG, get_tag = URL_HTML_TAG)
URLs = Append_Links(func = Select_Links_Starts_With,links = URLs, stems = URLS_START_WITH)
URLs = Remove_Links(func = Select_Links_Ends_With,links = URLs, stems = URLS_NOT_END_WITH)
Get_Text_From_Page(URLs[1])

'By  <a class=author href=homesearchhtml?s=&amp;authornamef=Charlie+Coë+For+Daily+Mail+Australia rel=nofollowCharlie Coë For Daily Mail Australia<a   <span class=article-timestamp article-timestamp-published <span class=article-timestamp-labelPublished:<span <time datetime=2021-11-16T20:57:42+0000 15:57 EST 16 November 2021 <time <span  |  <span class=article-timestamp article-timestamp-updated <span class=article-timestamp-labelUpdated:<span <time datetime=2021-11-16T21:26:13+0000 16:26 EST 16 November 2021 <time <span     103 View  <br  comments  A fed-up TV reporter has glared as a protester interrupted her live cross about Victorias controversial new pandemic bill with a sign mocking Daniel Andrews The protester stood directly behind Today reporter Izabella Staskowski as she crossed live to the studio from Victorian parliament on Wednesday morning The man held a sign showing the faces of the premier Victorian Chief Health Officer Brett Sutton and Prime Minister  <a href=newsscott-m

In [ ]:
bag_of_words = []
for url in URLs:
  bag_of_words.append(Get_Text_From_Page(url))


In [ ]:
titles = []
for url in URLs:
  titles.append(Get_Soup(url).find("title").contents[0])

In [ ]:
output = pd.DataFrame({"URLS":URLs,"Bag Of Words":bag_of_words,"Title":titles}).drop_duplicates()
output.to_csv(SCRAPE_OUTPUT_FILE)

In [ ]:
len(output)

144